# HMM POS tagger, NLP assignment 2, FHNW FS18

## Read and prepare data

In [77]:
# map Stuttgart-Tübingen tagset to universal tagset
stts_to_univ = {
'ADJA': 'ADJ',
'ADJD': 'ADJ',
'ADV': 'ADV',
'APPR': 'ADP',
'APPRART': 'ADP',
'APPO': 'ADP',
'APZR': 'ADP',
'ART': 'DET',
'CARD': 'NUM',
'FM': 'X',
'ITJ': 'X',
'KOUI': 'CONJ',
'KOUS': 'CONJ',
'KON': 'CONJ',
'KOKOM': 'CONJ',
'NN': 'NOUN',
'NE': 'NOUN',
'PDS': 'PRON',
'PDAT': 'PRON',
'PIS': 'PRON',
'PIAT': 'PRON',
'PIDAT': 'PRON',
'PPER': 'PRON',
'PPOSS': 'PRON',
'PPOSAT': 'PRON',
'PRELS': 'PRON',
'PRELAT': 'PRON',
'PRF': 'PRON',
'PWS': 'PRON',
'PWAT': 'PRON',
'PWAV': 'PRON',
'PAV': 'PRON',
'PTKZU': 'PRT',
'PTKNEG': 'PRT',
'PTKVZ': 'PRT',
'PTKANT': 'PRT',
'PTKA': 'PRT',
'TRUNC': 'X',
'VVFIN': 'VERB',
'VVIMP': 'VERB',
'VVINF': 'VERB',
'VVIZU': 'VERB',
'VVPP': 'VERB',
'VAFIN': 'VERB',
'VAIMP': 'VERB',
'VAINF': 'VERB',
'VAPP': 'VERB',
'VMFIN': 'VERB',
'VMINF': 'VERB',
'VMPP': 'VERB',
'XY': 'X',
'$,': '.',
'$.': '.',
'$(': '.'
}

with open("POS_German_train.txt", "r") as f:
    content = f.read()
    
words = []
tags = []

for pair in content.split(';'):
    splitted = pair.split('/')
    if (len(splitted) == 2):
        words.append(splitted[0].strip())
        tags.append(stts_to_univ[splitted[1].strip()])
        


In [78]:
print(set(tags))

{'NOUN', 'X', 'CONJ', 'NUM', 'ADJ', 'ADP', 'PRON', '.', 'PRT', 'VERB', 'DET', 'ADV'}
